# Importing my own Books from Goodreads export tool
Goodreads export using: https://www.goodreads.com/review/import


In [34]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/Desktop/Code/mybook-dashboard'

In [35]:
# Improt packages
import pandas as pd
import json 
import numpy as np

# Import functions from apps folder
# from apps.collect_data import *

pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 80)

In [36]:
mybooksgr = pd.read_csv("assets/goodreads_library_export.csv")
# mybooksgr = mybooksgr.rename(columns=lambda x: x.replace(' ', '_'))


In [5]:
# See async_googleapi for google api call code....
# seperated because I am working on improvements to it

In [33]:
# Merge dataframes 
mybooks = pd.merge(mybooksgr,
                     apimydf,
                     on='Title', 
                     suffixes = ('_Goodreads', '_GoogleBooks'), 
                     how='left')

# Data cleaning books df
- page count categories
- filter if book is read of not 

In [35]:
# Page count category variable

def categorize_pages(number_of_pages):
    if number_of_pages >= 100 and number_of_pages <= 249:
        return '100-249'
    elif number_of_pages >= 250 and number_of_pages <= 349:
        return '250-349'
    elif number_of_pages >= 350 and number_of_pages <= 449:
        return '350-449'
    elif number_of_pages >= 450 and number_of_pages <= 599:
        return '450-599'
    elif number_of_pages >= 600 and number_of_pages <= 749:
        return '600-749'
    elif number_of_pages >= 750 and number_of_pages <= 999:
        return '750-999'
    else:
        return '1000+'

# Apply the categorize_pages function to create the 'Page_Cat' column
mybooks['Page_Cat'] = mybooks['Number_of_Pages'].apply(categorize_pages)

# Define the desired order of categories
category_order = ['100-249', '250-349', '350-449', '450-599', '600-749', '750-999', '1000+']

# Convert the 'Page_Cat' column to a categorical variable with the specified order
mybooks['Page_Cat'] = pd.Categorical(mybooks['Page_Cat'], categories=category_order, ordered=True)


In [36]:
# drop duplicates
mybooks = mybooks.drop_duplicates(subset=['Title', 'Author'])

In [38]:
# Create year and quarter read variable 
#  Impute data_added where date_read  is na
mybooks['Date_Read'] = np.where(mybooks['Date_Read'].isnull() & mybooks['Read_Count']==1, mybooks['Date_Added'], mybooks['Date_Read'])

# Convert 'Date_Read' column to datetime type
mybooks['Date_Read'] = pd.to_datetime(mybooks['Date_Read'], format='mixed')

# Extract year and quarter from 'Date_Read' column
mybooks['Year'] = mybooks['Date_Read'].dt.year
mybooks['Quarter'] = mybooks['Date_Read'].dt.quarter

# Create a new column combining year and quarter
mybooks['Year_Quarter'] = np.where(mybooks['Date_Read'].notnull(), mybooks['Year'].astype(str) + '-Q' + mybooks['Quarter'].astype(str), np.nan)
# Replace '.0' in the Year_Quarter column with an empty string
mybooks['Year_Quarter'] = mybooks['Year_Quarter'].fillna('').str.replace('.0', '')

# Convert Year_Quarter to categorical variable
mybooks['Year_Quarter'] = pd.Categorical(mybooks['Year_Quarter'], ordered=True)

In [41]:
# filter na in publication year and make column publication year integer 
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].fillna( 0)
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].astype(int)

In [43]:
# Making sure all na is set as np.nan and not as a string variable (had this issue with one variable)
import numpy as np
mybooks = mybooks.replace('nan', np.nan)
mybooks = mybooks.replace('NaN', np.nan)

In [44]:
import pickle
mybooks.to_pickle("assets/my_books.pkl")

# Collect book topics 

In [6]:
# Collect topics for my own books from OLapi
from apps.collect_data import get_book_topics
my_topics = get_book_topics(mybooksgr.head(1))

In [21]:
dict = mybooksgr[['Author','Title']].to_dict()
list(set(dict['Title'].values()))
    

['A Court of Mist and Fury (A Court of Thorns and Roses, #2)',
 'A Long Petal of the Sea',
 'Animal Farm / 1984',
 'Colorless Tsukuru Tazaki and His Years of Pilgrimage',
 'The Girl on the Train',
 'Hidden Pictures',
 'The History of Bees (Klimakvartetten, #1)',
 'Saving Grace',
 'Kim Jiyoung, Born 1982',
 'The Data Detective: Ten Easy Rules to Make Sense of Statistics',
 'Lonely Castle in the Mirror',
 'Everything is Illuminated',
 'Dominicana',
 'Berlinerpoplene (Familien Neshov, #1)',
 'Behold the Dreamers',
 'Bellies',
 'De syv søstre',
 'Killing Commendatore',
 'And Then There Were None',
 'Lucifers evangelium (Bjørn Beltø, #3)',
 'Ghosts',
 'Memphis',
 'The Midnight Rose',
 'The Murderbot Diaries #1-4: : All Systems Red, Artificial Condition, Rogue Protocol, Exit Strategy',
 'My Brilliant Friend (The Neapolitan Novels, #1)',
 'Fiesta: The Sun Also Rises',
 'A Court of Thorns and Roses (A Court of Thorns and Roses, #1)',
 'None of This Is True',
 'The Song of Achilles',
 'Things W

In [14]:
df = pd.DataFrame(dict)
df

,Author,Title
0,Rebecca James,Beautiful Malice
1,Lily King,Writers & Lovers
2,Lucy Foley,The Guest List
3,Nita Prose,"The Mystery Guest (Molly the Maid, #2)"
4,Anne B. Ragde,"Berlinerpoplene (Familien Neshov, #1)"
...,...,...
397,Ken Follett,"The Evening and the Morning (Kingsbridge, #0)"
398,Anthony Doerr,All the Light We Cannot See
399,George Orwell,1984
400,Cristina García,Monkey Hunting: A Novel (Ballantine Reader's C...


In [22]:
# Writes the topics as 
with open("assets/my_topics.json", "w") as outfile:
    json.dump(my_topics, outfile)

In [41]:
mybooksgr.query('`Read Count` ==1').sample(2).to_csv("assets/my_books_test.csv", index=False)